#### Lesson Objective:
* learn how to read a video in Python
* learn how to separate background from moving objects using Non-negative Matrix Factorization

In [ ]:
# import common modules
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

#### Reading Video

In [ ]:
# set the path
video_path = 'data/MAH08632.MP4'

Unfortunately, Anaconda installation does not provide a package which reads `.mp4` directly. (`scikit-image` can read `.tiff` sequences). A useful package to do that is `imageio` which can be installed separately:
    
```
conda install -c menpo imageio=1.5.0
```

In [ ]:
# import and test the presence of 'ffmpeg'
import imageio
imageio.plugins.ffmpeg.download()

In [ ]:
# get a pointer to the video
vid = imageio.get_reader(video_path,'ffmpeg')

In [ ]:
# extract and display the first frame
image = vid.get_data(0)
plt.imshow(image)
m = image.shape[0]
n = image.shape[1]
nofFrames = len(vid)

We will not read the whole video as it will overfill the RAM. We will subsample by 50. We will also read just the first channel, and stack all the pixels in an vector.

In [ ]:
# sequence of frames to read
nums = np.linspace(0,nofFrames-1,50)

# read the video frameas and store in a list
video = []
for num in nums:
    video.append(vid.get_data(num)[:,:,0].ravel())

In [ ]:
# convert the list to array
data = np.array(video)

In [ ]:
data.shape

#### Background Separation via NMF

In [ ]:
# we hope that the background will correspond to the first component
n_components = 1


# perform NMF
from sklearn.decomposition import NMF
model = NMF(init="nndsvd", n_components=n_components, random_state=1)
W_matrix = model.fit_transform(data)
H_matrix = model.components_

In [ ]:
plt.figure(figsize = (7,7))
plt.imshow(H_matrix.reshape(m,n,n_components)[:,:,0],cmap = 'gray')
plt.axis('off')

#### Tips for working with large videos:
* consider memory mapping the video to a file on disk when the video cannot fit in RAM 
* store them as `.hdf5` to allow quick individual access to frames
* use out-of-core libraries such as `dask.array` to do operations on individual chunks and combine the results